In [1]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.classification import GBTClassifier
from pyspark.ml.classification import NaiveBayes

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import IndexToString, StringIndexer, VectorIndexer
from pyspark.ml import Pipeline

from pyspark.sql import functions as F

import math

algos=["RF","ANN","DT","GBT"]

ImportError: No module named pyspark.ml.classification

In [2]:
def parsePoint(line):
    data=line.split(',')
    values = [float(data[x]) for x in range(0,len(data)-1)]
    #Tr
    #values.append(values[5]-values[4])
    #Tr
    #values.append(values[7]-values[4])
    #Tr
    #values.append(values[16]-values[4])
    
    values.append(0 if (data[len(data)-1] == 'N') else 1)
    return (values[len(values)-1],Vectors.dense(values[0:len(values)-1]))
    

def confCalc(a):
    ideal,pred=a.label,a.prediction
    tp=fp=tn=fn=0.0
    if(ideal == 1 and pred == 1):
        tp=1.0
    if(ideal == 1 and pred == 0):
        fn=1.0
    if(ideal == 0 and pred == 0):
        tn=1.0
    if(ideal == 0 and pred == 1):
        fp=1.0
    
    return(ideal,pred,tp,fp,tn,fn)

def paramCalc(cf):
    cf=sqlContext.createDataFrame(cf,['ideal','predicted','tp','fp','tn','fn'])
    add=cf.agg(F.sum(cf.tp).alias('tp'),F.sum(cf.fp).alias('fp'),F.sum(cf.tn).alias('tn'),\
           F.sum(cf.fn).alias('fn'),\
           (F.sum(cf.fp)/(F.sum(cf.fp)+F.sum(cf.tn))).alias('fpr'),\
           (F.sum(cf.tp)/(F.sum(cf.tp)+F.sum(cf.fn))).alias('tpr'),\
           (F.sum(cf.tn)/(F.sum(cf.fp)+F.sum(cf.tn))).alias('tnr'),\
           (F.sum(cf.fn)/(F.sum(cf.tp)+F.sum(cf.fn))).alias('fnr'),\
           (F.sum(cf.tp)/(F.sum(cf.tp)+F.sum(cf.fn))).alias('recall'),\
           (F.sum(cf.tp)/(F.sum(cf.tp)+F.sum(cf.fp))).alias('precision'),\
           ((F.sum(cf.tp)+F.sum(cf.tn))/(F.sum(cf.tp)+F.sum(cf.fn)+F.sum(cf.tn)+F.sum(cf.fp))).alias('accuracy'),\
           (2*F.sum(cf.tp)/(2*F.sum(cf.tp)+F.sum(cf.fp)+F.sum(cf.fn))).alias('Fmeasure'),\
           (1-((F.sum(cf.tp)+F.sum(cf.tn))/(F.sum(cf.tp)+F.sum(cf.fn)+F.sum(cf.tn)+F.sum(cf.fp)))).alias('error'),\
          )
    return add

def nbExec(test):
    predictions = nbModel.transform(test)
    nbResult=predictions.select("label","prediction")
    return nbResult

def lrExec(test):
    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)
    lrModel = lr.fit(test)
    trainingSummary = lrModel.summary
    lrResult=trainingSummary.predictions.select("label","prediction")
    return lrResult

def dtExec(test):
    predictions = dtModel.transform(test)
    dtResult=predictions.select("label","prediction")
    return dtResult
    
def rfExec(test):
    predictions = rfModel.transform(test)
    rfResult=predictions.select("label", "prediction")
    return rfResult
    
def gbtExec(test):
    predictions = gbtModel.transform(test)
    gbtResult=predictions.select("label","prediction")
    return gbtResult
    
def annExec(test):
    result = annModel.transform(test)
    annResult = result.select( "label","prediction")
    return annResult

def run(technique):
    if (technique=='LR'):
        result=lrExec(test)
    elif (technique=='NB'):
        result=nbExec(test)
    elif (technique=='DT'):
        result=dtExec(test)
    elif (technique=='RF'):
        result=rfExec(test)
    elif (technique=='GBT'):
        result=gbtExec(test)
    elif (technique=='ANN'):
        result=annExec(test)
        
    return result,testData.select('id').rdd.collect()
        
def profitCalc(predictions):
    
    metricDF=sqlContext.createDataFrame([predictions],["Technique","tp","fp","tn","fn","fpr","tpr","tnr","fnr","recall","precision","accuracy","fmeasure","error"]) 
    
    tp=metricDF.first()['tp']
    fp=metricDF.first()['fp']
    tn=metricDF.first()['tn']
    fn=metricDF.first()['fn']
    
    gw=1
    amount=100
    
    profit=(fp*gw)+(fn*amount)+(fn*gw/2.0)
    
    return (metricDF.select('Technique').rdd.collect()[0]['Technique'],profit)
    
def profitCalc1(predictions):
    gamma=0.3
    CLV=200
    co=10
    ca=1
    
    metricDF=sqlContext.createDataFrame([predictions],["Technique","tp","fp","tn","fn","fpr","tpr","tnr","fnr","recall","precision","accuracy","fmeasure","error"]) 
    
    profit=metricDF.first()['tp']*(gamma*(CLV+co+ca)+((1-gamma)*ca))+(metricDF.first()['fp']*(-co-ca))
    
    return (metricDF.select('Technique').rdd.collect()[0]['Technique'],profit)

def getModel(technique):
    if (technique=='LR'):
        result=lrModel.transform(test)
    elif (technique=='NB'):
        result=nbModel.transform(test)
    elif (technique=='DT'):
        result=dtModel.transform(test)
    elif (technique=='RF'):
        result=rfModel.transform(test)
    elif (technique=='GBT'):
        result=gbtModel.transform(test)
    elif (technique=='ANN'):
        result=annModel.transform(test)
    
    return result

# Models

In [3]:
def LR():
    global lrModel
    
    lr = LogisticRegression(maxIter=10, regParam=0.3, elasticNetParam=0.8)

    # Fit the model
    lrModel = lr.fit(train.select('label','features'))

    # Extract the summary from the returned LogisticRegressionModel instance trained
    # in the earlier example
    trainingSummary = lrModel.summary

    lrResult=trainingSummary.predictions.select("label","prediction")
    
    return lrResult

def DT():
    global dtModel
    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

    # Automatically identify categorical features, and index them.
    # We specify maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

    # Split the data into training and test sets (30% held out for testing)

    # Train a DecisionTree model.
    dt = DecisionTreeClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures")

    # Chain indexers and tree in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, dt])

    # Train model.  This also runs the indexers.
    dtModel = pipeline.fit(train)

    # Make predictions.
    predictions = dtModel.transform(test)

    dtResult=predictions.select("label","prediction")
    
    return dtResult

def RF():
    global rfModel
    
    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)

    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)


    # Train a RandomForest model.
    rf = RandomForestClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", numTrees=10)

    # Convert indexed labels back to original labels.
    labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel",
                                   labels=labelIndexer.labels)

    # Chain indexers and forest in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, rf, labelConverter])

    # Train model.  This also runs the indexers.
    rfModel = pipeline.fit(train)

    # Make predictions.
    predictions = rfModel.transform(test)

    rfResult=predictions.select("label", "prediction")
    
    return rfResult

def ANN():
    global annModel
    
    
    # specify layers for the neural network:
    #Count of features alone in input layer
    #No of output classes in output layer

    #layers = [3, 6,7, 2]
    layers = [17, 18, 2]

    # create the trainer and set its parameters
    trainer = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)

    # train the model
    annModel = trainer.fit(train)

    # compute accuracy on the test set
    result = annModel.transform(test)
    annResult = result.select( "label","prediction")
    return annResult

def GBT():
    global gbtModel
    # Index labels, adding metadata to the label column.
    # Fit on whole dataset to include all labels in index.
    labelIndexer = StringIndexer(inputCol="label", outputCol="indexedLabel").fit(data)
    # Automatically identify categorical features, and index them.
    # Set maxCategories so features with > 4 distinct values are treated as continuous.
    featureIndexer =\
        VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=4).fit(data)

    # Train a GBT model.
    gbt = GBTClassifier(labelCol="indexedLabel", featuresCol="indexedFeatures", maxIter=10)

    # Chain indexers and GBT in a Pipeline
    pipeline = Pipeline(stages=[labelIndexer, featureIndexer, gbt])

    # Train model.  This also runs the indexers.
    gbtModel = pipeline.fit(train)

    # Make predictions.
    predictions = gbtModel.transform(test)

    gbtResult=predictions.select("label","prediction")
    
    return gbtResult

def NB():
    global nbModel
    
    # create the trainer and set its parameters
    nb = NaiveBayes(smoothing=1.0, modelType="multinomial")

    # train the model
    nbModel = nb.fit(train)

    # select example rows to display.
    predictions = nbModel.transform(test)
    nbResult=predictions.select("label","prediction")
    
    return nbResult

In [4]:
data = sc.textFile("bankData.csv")
parsedData = data.map(parsePoint)
data=sqlContext.createDataFrame(parsedData,['label','features'])

(train, test) = data.randomSplit([0.7, 0.3])

NameError: name 'sc' is not defined

In [5]:
techniqueCount=4

#lrCM=LR().rdd.map(confCalc)
rfCM=RF().rdd.map(confCalc)
annCM=ANN().rdd.map(confCalc)
dtCM=DT().rdd.map(confCalc)
gbtCM=GBT().rdd.map(confCalc)
#nbCM=NB().rdd.map(confCalc)

In [17]:
metricsDF=[]
#metricsDF.append(paramCalc(lrCM))
metricsDF.append(paramCalc(rfCM))
metricsDF.append(paramCalc(annCM))
metricsDF.append(paramCalc(dtCM))
metricsDF.append(paramCalc(gbtCM))
#metricsDF.append(paramCalc(nbCM))

metricResults=[]

count=0

for results in metricsDF:
    tempMetrics=[algos[count]]
    a=results.rdd.collect()
    for x in a[0]:
        tempMetrics.append(x)
    
    metricResults.append(tempMetrics)
    count+=1
    

In [18]:
#RULE BUILDING

metrics=sqlContext.createDataFrame(metricResults,["Technique","tp","fp","tn","fn","fpr","tpr","tnr","fnr","recall","precision","accuracy","fmeasure","error"])
#Techniques with best TPR (a)

trueSelection=metrics.orderBy(metrics.tpr.desc()).limit(1)
    
#Techniques with best TNR (b)
falseSelection=metrics.orderBy(metrics.tnr.desc()).limit(1)
    
selectedAlgos=falseSelection.union(trueSelection).rdd.collect()
#selectedAlgos=selectedAlgos.distinct().orderBy(selectedAlgos.fmeasure).rdd.collect()


In [19]:
probabilityLevels=[]

for trueAlgos in selectedAlgos:
    #Contains all predictions
    #print trueAlgos
    probabilityLevels.append(getModel(trueAlgos['Technique']).select('prediction').rdd.collect())
    

probabilityList={}
for algoIndex,algo in enumerate(probabilityLevels):
    print "Algo Index:"+str(algoIndex)
    for index,pred in enumerate(algo):
        if algoIndex==0:
            probabilityList[index]=pred['prediction']
        else:
            if pred['prediction']==1:
                probabilityList[index]=1.0


labelList=test.select('label').rdd.collect()
finalPred=[]
for index in probabilityList:
    finalPred.append([labelList[index]['label'],probabilityList[index]])
    
finalDF=sqlContext.createDataFrame(finalPred,['label','prediction'])

#Find Confusion Matrix

finalResult=finalDF.select('label','prediction').rdd.map(confCalc)

#Find Metrics

finalMetrics=paramCalc(finalResult)

predMetrics=finalMetrics.rdd.collect()[0]

finalMetrics=['Proposed']


for x in predMetrics:
    finalMetrics.append(x)
    
finalResults=metricResults
finalResults.append(finalMetrics)

for algo in finalResults:
    prof=profitCalc(algo)
    algo.append(prof[1])

Algo Index:0
Algo Index:1


In [20]:
f=open('BankResults.csv','w')

f.write("\nTechnique,tp,fp,tn,fn,fpr,tpr,tnr,fnr,recall,precision,accuracy,fmeasure,error,loss")
print("Technique,tp,fp,tn,fn,fpr,tpr,tnr,fnr,recall,precision,accuracy,fmeasure,error,loss")

for metrics in finalResults:
    line="\n"
    for m in metrics:
        line+=str(m)+","
        
    f.write(line)
    print(line)
    
f.close()




Technique,tp,fp,tn,fn,fpr,tpr,tnr,fnr,recall,precision,accuracy,fmeasure,error,loss

RF,86.0,14.0,100121.0,3737.0,0.000139811254806,0.0224954224431,0.999860188745,0.977504577557,0.0224954224431,0.86,0.96391812078,0.0438439969411,0.0360818792205,375582.5,

ANN,623.0,428.0,99707.0,3200.0,0.00427422978978,0.162961025373,0.99572577021,0.837038974627,0.162961025373,0.592768791627,0.965101290906,0.255642183012,0.0348987090941,322028.0,

DT,553.0,350.0,99785.0,3270.0,0.00349528137015,0.144650797803,0.99650471863,0.855349202197,0.144650797803,0.612403100775,0.965178245061,0.23402454507,0.0348217549395,328985.0,

GBT,826.0,360.0,99775.0,2997.0,0.00359514655215,0.216060685326,0.996404853448,0.783939314674,0.216060685326,0.696458684654,0.967708112892,0.329806348573,0.0322918871083,301558.5,

Proposed,842.0,366.0,99769.0,2981.0,0.00365506566136,0.220245880199,0.996344934339,0.779754119801,0.220245880199,0.69701986755,0.967804305585,0.334724706818,0.0321956944151,299956.5,
